Imports and setting global seed:

In [1]:
%pip install lightning_lite
import numpy as np
import anndata as ad
import pandas as pd
import torch
from lightning_lite import seed_everything
from pytorch_lightning.callbacks import EarlyStopping  # ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import make_classification
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score

seed_everything(10, workers=True)

Note: you may need to restart the kernel to use updated packages.


c:\Users\miria\anaconda3\envs\dl4ls\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 10


10

Hyperparameters:

In [25]:
base_score=0.5
colsample_bylevel=1
colsample_bytree=1
gamma=0
learning_rate=0.1
max_depth=3
min_child_weight=1
seed=10



Loading data from path, converting objs to a dataframe for analysis and prediction task

In [3]:
path="C:/Users/miria/OneDrive/Documents/AMatematyka/aaaaDL4LS/DL4LS24-IDK-main/data/train/cell_data.h5ad"
### input Your path

anndata = ad.read_h5ad(path)
anndata.layers
anndata.X = anndata.layers['exprs'] # FIX!

df=anndata.obs
counts=anndata.layers['counts']
exprs=anndata.layers['exprs']

In [4]:
df.shape

(236791, 39)

Appending information from counts and exprs to dataframe for prediction task

In [5]:
list_cols_exprs=['exprs'+str(i) for i in range(40)]
df_exprs = pd.DataFrame(data=exprs, columns=list_cols_exprs,index=df.index)

In [6]:
df1=df.copy()
merged_df = pd.concat([df1, df_exprs], axis=1)
print(merged_df.shape)


df=merged_df.copy()

(236791, 79)


TESTTT ---wyciete dane z obs, komorka do usuniecia raczej, pokazuje ze mamy dobre wartosci na samych counts i exprs

In [242]:
'''df_test=pd.concat([df_exprs, df_counts], axis=1)
y=df['cell_labels']

encoder = LabelEncoder()
y_encoded1 = encoder.fit_transform(y)

seed = 10
test_size=0.2
X_train, X_test, Y_train, Y_test = train_test_split(df_test, y_encoded1, test_size=test_size, random_state=seed)

xgb_clf1 = xgb.XGBClassifier()
xgb_clf1 = xgb_clf1.fit(X_train, Y_train)

predictions=xgb_clf1.predict(X_test)
predictions_probabilities=xgb_clf1.predict_proba(X_test)

accuracy = accuracy_score(Y_test, predictions)
print("Accuracy:", accuracy)

macro_f1 = f1_score(Y_test, predictions, average='macro')
print("Macro F1 score:", macro_f1)

auc = roc_auc_score(Y_test, predictions_probabilities, multi_class='ovr')
print("AUC score (OvR):", auc)

average_precision = average_precision_score(Y_test, predictions_probabilities)
print("Average Precision score:", average_precision)'''

Accuracy: 0.9719799826854452
Macro F1 score: 0.9458052709988252
AUC score (OvR): 0.9994492257959756
Average Precision score: 0.9839467390880151


Data preprocessing; 
1. Remove unneccesary columns 
2. One-hot encode categorical ones 
3. End up with only float values

In [7]:
print("All columns: ",df.columns)
int_columns = df.select_dtypes(include='int').columns.tolist()
df[int_columns] = df[int_columns].astype(float)

non_float_columns = df.select_dtypes(exclude='float').columns.tolist()
'''for column in non_float_columns:
    print(f"{column}: {df[column].dtype}")'''
category_columns = df.select_dtypes(include='category').columns.tolist()
print(df['sample_id'].equals(df['SampleId']))
ids=df['sample_id']
Y=df['cell_labels']

columns_to_remove=['SampleId','image','sample_id','SlideId','BatchId','SubBatchId','Batch','Box.Description','cell_labels','celltypes']

df = df.drop(columns=columns_to_remove)
df['CD20_patches'] = df['CD20_patches'].replace('', '0')
df['CD20_patches'] = df['CD20_patches'].astype(float)

category_columns = df.select_dtypes(include='category').columns.tolist()
category_columns

one_hot_encoded1 = pd.get_dummies(df['Indication'], prefix='Indication')
one_hot_encoded1
one_hot_encoded1= one_hot_encoded1.astype(float)
df = df.join(one_hot_encoded1)

one_hot_encoded2 = pd.get_dummies(df['Study'], prefix='Study')
one_hot_encoded2= one_hot_encoded2.astype(float)
one_hot_encoded2
df = df.join(one_hot_encoded2)
df = df.drop(columns=['Indication','Study'])
category_columns = df.select_dtypes(include='category').columns.tolist()

df.dtypes

All columns:  Index(['image', 'sample_id', 'ObjectNumber', 'Pos_X', 'Pos_Y', 'area',
       'major_axis_length', 'minor_axis_length', 'eccentricity', 'width_px',
       'height_px', 'acquisition_id', 'SlideId', 'Study', 'Box.Description',
       'Position', 'SampleId', 'Indication', 'BatchId', 'SubBatchId', 'ROI',
       'ROIonSlide', 'includeImage', 'flag_no_cells', 'flag_no_ROI',
       'flag_total_area', 'flag_percent_covered', 'small_cell', 'celltypes',
       'flag_tumor', 'PD1_pos', 'Ki67_pos', 'cleavedPARP_pos', 'GrzB_pos',
       'tumor_patches', 'distToCells', 'CD20_patches', 'Batch', 'cell_labels',
       'exprs0', 'exprs1', 'exprs2', 'exprs3', 'exprs4', 'exprs5', 'exprs6',
       'exprs7', 'exprs8', 'exprs9', 'exprs10', 'exprs11', 'exprs12',
       'exprs13', 'exprs14', 'exprs15', 'exprs16', 'exprs17', 'exprs18',
       'exprs19', 'exprs20', 'exprs21', 'exprs22', 'exprs23', 'exprs24',
       'exprs25', 'exprs26', 'exprs27', 'exprs28', 'exprs29', 'exprs30',
       'exprs31', 

C:\Users\miria\AppData\Local\Temp\ipykernel_25284\1152296002.py:16: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df['CD20_patches'] = df['CD20_patches'].replace('', '0')


ObjectNumber          float64
Pos_X                 float64
Pos_Y                 float64
area                  float64
major_axis_length     float64
                       ...   
Study_180311_GI       float64
Study_180327_HN       float64
Study_180345_BREAS    float64
Study_190346_GU       float64
Study_190370_SPECT    float64
Length: 80, dtype: object

Dataframe df is now filled with float values, Y is our label derived from cell_labels column, we encode it below:

In [9]:
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(Y)
num_classes = len(np.unique(y_encoded))
num_classes

14

X is our data which is going to be fed to the model:

In [10]:
X=df.copy()
seed = 10
test_size=0.2
X_train, X_test, Y_train, Y_test = train_test_split(X, y_encoded, test_size=test_size, random_state=seed)


We train XGboost model on a train set (should take 2mins on 0.8 of data)

In [26]:
xgb_clf = xgb.XGBClassifier(base_score=base_score, colsample_bylevel=colsample_bylevel, colsample_bytree=colsample_bytree,max_depth=max_depth,
                             min_child_weight=min_child_weight,gamma=gamma, learning_rate=learning_rate, seed=10)

xgb_clf = xgb_clf.fit(X_train, Y_train)

We make predictions for test set and calculate metrics:

In [27]:
predictions=xgb_clf.predict(X_test)
predictions_probabilities=xgb_clf.predict_proba(X_test)

In [28]:
accuracy = accuracy_score(Y_test, predictions)
print("Accuracy:", accuracy)

macro_f1 = f1_score(Y_test, predictions, average='macro')
print("Macro F1 score:", macro_f1)

auc = roc_auc_score(Y_test, predictions_probabilities, multi_class='ovr')
print("AUC score (OvR):", auc)

average_precision = average_precision_score(Y_test, predictions_probabilities)
print("Average Precision score:", average_precision)

Accuracy: 0.9655186976076353
Macro F1 score: 0.9271945217141659
AUC score (OvR): 0.9989213301507792
Average Precision score: 0.9715489976059903


Final model trained on the whole set

In [29]:
xgb_clf = xgb.XGBClassifier(base_score=base_score, colsample_bylevel=colsample_bylevel, colsample_bytree=colsample_bytree,max_depth=max_depth,
                             min_child_weight=min_child_weight,gamma=gamma, learning_rate=learning_rate, seed=10)

xgb_clf = xgb_clf.fit(X, y_encoded)

In [ ]:
new_input='...'
new_output='...'

predictions=xgb_clf.predict(new_input)
predictions_probabilities=xgb_clf.predict_proba(new_input)

In [ ]:
accuracy = accuracy_score(new_output, predictions)
print("Accuracy:", accuracy)

macro_f1 = f1_score(new_output, predictions, average='macro')
print("Macro F1 score:", macro_f1)

auc = roc_auc_score(new_output, predictions_probabilities, multi_class='ovr')
print("AUC score (OvR):", auc)

average_precision = average_precision_score(new_output, predictions_probabilities)
print("Average Precision score:", average_precision)